In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [2]:
a_pos_5289 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana5289_pos/kmer3/A.thaliana5289_pos_kmer_tnc_3.npy"
a_neg_5289 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana5289_neg/kmer3/A.thaliana5289_neg_kmer_tnc.npy"
a_pos_1000 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana1000indep_pos/kmer3/A.thaliana1000indep_pos_kmer_tnc.npy"
a_neg_1000 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana1000indep_neg/kmer_tnc/A.thaliana1000indep_neg_kmer_tnc.npy"

a_x_train_pos = np.load(a_pos_5289) 
a_x_train_neg = np.load(a_neg_5289) 
a_x_test_pos = np.load(a_pos_1000) 
a_x_test_neg = np.load(a_neg_1000) 


b_pos_5289 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana5289_pos/pseknc/A.thaliana5289_pos_pseknc.npy"
b_neg_5289 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana5289_neg/pseknc/A.thaliana5289_neg_pseknc.npy"
b_pos_1000 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana1000indep_pos/pseknc/A.thaliana1000indep_pos_pseknc.npy"
b_neg_1000 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana1000indep_neg/pseknc/A.thaliana1000indep_neg_pseknc.npy"

b_x_train_pos = np.load(b_pos_5289) 
b_x_train_neg = np.load(b_neg_5289) 
b_x_test_pos = np.load(b_pos_1000) 
b_x_test_neg = np.load(b_neg_1000) 

y_train_pos = np.tile(1, 5289) 
y_train_neg = np.tile(0, 5289) 
y_test_pos = np.tile(1, 1000) 
y_test_neg = np.tile(0, 1000) 

In [4]:
print(a_x_test_neg.shape)
print(a_x_train_neg.shape)

print(a_x_test_pos.shape)
print(a_x_train_pos.shape)

print()
print(b_x_test_neg.shape)
print(b_x_train_neg.shape)

print(b_x_test_pos.shape)
print(b_x_train_pos.shape)

(1000, 64)
(5289, 64)
(1000, 64)
(5289, 64)

(1000, 66)
(5289, 66)
(1000, 66)
(5289, 66)


In [7]:
a_x_training = np.concatenate((a_x_train_pos, a_x_train_neg), axis = 0)
a_x_test = np.concatenate((a_x_test_pos, a_x_test_neg), axis = 0)

b_x_training = np.concatenate((b_x_train_pos, b_x_train_neg), axis = 0)
b_x_test = np.concatenate((b_x_test_pos, b_x_test_neg), axis = 0)

y_train = np.concatenate((y_train_pos,y_train_neg))
y_test = np.concatenate((y_test_pos,y_test_neg))

In [8]:
print(a_x_training.shape)
print(a_x_test.shape)

print(b_x_training.shape)
print(b_x_test.shape)

print()
print(y_test.shape)
print(y_train.shape)



(10578, 64)
(2000, 64)
(10578, 66)
(2000, 66)

(2000,)
(10578,)


In [9]:
x_training = np.concatenate((a_x_training, b_x_training), axis = 1)

x_test = np.concatenate((a_x_test, b_x_test), axis = 1)

In [10]:
print(x_training.shape)
print(x_test.shape)

(10578, 130)
(2000, 130)


In [13]:
seed = 40
np.random.seed(seed)

kf = KFold(n_splits=5, random_state=seed, shuffle=True)

In [14]:
test_avg_acc_list = []
j = 1
for c in range(1,100000,10000):
  clf = SVC(C=c,  random_state = seed)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  y_pred_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    clf.fit(X_train, y_train)

    y_val_pred = clf.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = clf.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = clf.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
    y_pred_list.append(y_test_pred)
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 1
Train Accuracy :  1.0 	Validation Accuracy :  0.998109640831758 	Test Accuracy  0.6795
Train Accuracy :  1.0 	Validation Accuracy :  0.997164461247637 	Test Accuracy  0.677
Train Accuracy :  1.0 	Validation Accuracy :  0.9966918714555766 	Test Accuracy  0.681
Train Accuracy :  1.0 	Validation Accuracy :  0.9985815602836879 	Test Accuracy  0.687
Train Accuracy :  1.0 	Validation Accuracy :  0.9976359338061466 	Test Accuracy  0.6765
Avg Test Accuracy: 0.6801999999999999
Epoch 2/10 for C = 10001
Train Accuracy :  1.0 	Validation Accuracy :  0.999054820415879 	Test Accuracy  0.6765
Train Accuracy :  1.0 	Validation Accuracy :  0.9976370510396976 	Test Accuracy  0.678
Train Accuracy :  1.0 	Validation Accuracy :  0.999054820415879 	Test Accuracy  0.6875
Train Accuracy :  1.0 	Validation Accuracy :  0.9990543735224586 	Test Accuracy  0.687
Train Accuracy :  1.0 	Validation Accuracy :  0.9981087470449173 	Test Accuracy  0.6755
Avg Test Accuracy: 0.6809000000000001
Epoch 3